<a href="https://colab.research.google.com/github/dogee4803/db_course/blob/main/DB_Week9_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Setting up the Environment and Importing Necessary Libraries
!pip install ipython-sql
import sqlite3
import pandas as pd
%load_ext sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00


In [ ]:
# Step 2: Creating a Database
conn = sqlite3.connect('lab_session.db')
cursor = conn.cursor()

In [ ]:
import sqlite3

# Step 3: Creating Tables and Populating them
conn = sqlite3.connect('lab_session.db')
c = conn.cursor()
commands = [
    """CREATE TABLE IF NOT EXISTS students (
      student_id INTEGER PRIMARY KEY,
      first_name TEXT,
      last_name TEXT,
      major TEXT
    );
    """,
    """
    INSERT INTO students (first_name, last_name, major)
    VALUES ('John', 'Doe', 'Computer Science'),
           ('Jane', 'Doe', 'Mathematics'),
           ('Mike', 'Jordan', 'Physics');
    """,
    """
    CREATE TABLE IF NOT EXISTS courses (
      course_id INTEGER PRIMARY KEY,
      course_name TEXT,
      department_id INTEGER,
      instructor_id INTEGER,
      FOREIGN KEY (department_id) REFERENCES departments(department_id),
      FOREIGN KEY (instructor_id) REFERENCES instructors(instructor_id)
    );
    """,
    """
    INSERT INTO courses (course_name, department_id, instructor_id)
    VALUES ('Database Systems', 1 , 1),
           ('Data Structures', 1 , 2),
           ('Linear Algebra', 2 , 3) ,
           ('Quantum Mechanics', 3, 4);
    """,
    """
   CREATE TABLE IF NOT EXISTS enrollments (
      enrollment_id INTEGER PRIMARY KEY,
      student_id INTEGER,
      course_id INTEGER,
      FOREIGN KEY (student_id) REFERENCES students (student_id),
      FOREIGN KEY (course_id) REFERENCES courses (course_id)
   );
   """,
   """
   INSERT INTO enrollments (student_id, course_id )
   VALUES (1, 1),
          (1, 2),
          (2, 3),
          (3, 4);
   """,
   """
   CREATE TABLE IF NOT EXISTS instructors (
      instructor_id INTEGER PRIMARY KEY,
      first_name TEXT,
      last_name TEXT,
      department_id INTEGER,
      FOREIGN KEY (department_id) REFERENCES departments (department_id)
   );
   """,
   """
   INSERT INTO instructors (first_name, last_name, department_id)
   VALUES ('Robert', 'Smith', 1),
   ('Susan', 'Johnson', 2),
   ('Michael', 'Brown', 3),
   ('Elizabeth', 'Taylor', 4);
   """ ,
   """
   CREATE TABLE IF NOT EXISTS departments (
      department_id INTEGER PRIMARY KEY,
      department_name TEXT
   );
   """,
   """
   INSERT INTO departments (department_name)
   VALUES ('Computer Science'),
          ('Mathematics'),
          ('Physics'),
          ('Chemistry');
   """
]
# Execute SQL commands
for command in commands:
  c.execute(command)

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect ('lab_session.db')

# Execute INNER JOIN query and visualize the result
inner_join_query = """
    SELECT students.first_name AS student_name, courses.course_name
    FROM students
    INNER JOIN enrollments ON students.student_id = enrollments.student_id
    INNER JOIN courses ON enrollments.course_id = courses.course_id;
    """

inner_join_df = pd.read_sql_query(inner_join_query, conn)
inner_join_df


,student_name,course_name
0,John,Database Systems
1,John,Data Structures
2,Jane,Linear Algebra
3,Mike,Quantum Mechanics


In [ ]:
# Execute LEFT JOIN query and visualize the result
left_join_query = """
    SELECT students.first_name AS student_name, courses.course_name
    FROM students
    LEFT JOIN enrollments ON students.student_id = enrollments.student_id
    LEFT JOIN courses ON enrollments.course_id = courses.course_id;
    """

left_join_df = pd.read_sql_query (left_join_query, conn )
left_join_df

,student_name,course_name
0,John,Database Systems
1,John,Data Structures
2,Jane,Linear Algebra
3,Mike,Quantum Mechanics


In [ ]:
# Reconnect to the SQLite database
conn = sqlite3.connect ('lab_session.db')
# Execute FULL JOIN query ( emulated ) and visualize the result
full_join_query_1 = """
WITH RECURSIVE full_join AS (
    SELECT students.first_name AS student_name, courses.course_name
    FROM students
    LEFT JOIN enrollments ON students.student_id = enrollments.student_id
    LEFT JOIN courses ON enrollments.course_id = courses.course_id
    UNION ALL
    SELECT students.first_name AS student_name, courses.course_name
    FROM courses
    LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
    LEFT JOIN students ON enrollments.student_id = students.student_id
    WHERE students.student_id IS NULL
    )
SELECT * FROM full_join ;
"""

full_join_df_1 = pd.read_sql_query (full_join_query_1, conn)
full_join_df_1

,student_name,course_name
0,John,Database Systems
1,John,Data Structures
2,Jane,Linear Algebra
3,Mike,Quantum Mechanics


In [ ]:
# Reconnect to the SQLite database
conn = sqlite3 . connect ('lab_session.db')

# Execute Subquery and visualize the result
subquery_example_1 = """
    SELECT first_name, last_name
    FROM students
    WHERE student_id IN (
        SELECT student_id
        FROM enrollments
        WHERE course_id IN (
            SELECT course_id
            FROM courses
            WHERE course_name = 'Database Systems'
    )
);
"""

subquery_df_1 = pd.read_sql_query (subquery_example_1, conn)
subquery_df_1

,first_name,last_name
0,John,Doe


In [ ]:
# Execute Subquery and visualize the result
subquery_example_2 = """
    SELECT course_name
    FROM courses
    WHERE department_id = (
        SELECT department_id
        FROM departments
        WHERE department_name = 'Computer Science'
);
"""

subquery_df_2 = pd.read_sql_query(subquery_example_2, conn)
subquery_df_

,course_name
0,Database Systems
1,Data Structures


In [ ]:
# Execute Aggregate Function query and visualize the result
agg_function_example_1 = """
    SELECT departments.department_name, COUNT ( DISTINCT instructors.instructor_id ) AS instructor_count
    FROM departments
    LEFT JOIN instructors ON departments.department_id = instructors.department_id
    GROUP BY departments.department_name;
"""

agg_df_1 = pd.read_sql_query (agg_function_example_1, conn)
agg_df_1

,department_name,instructor_count
0,Chemistry,1
1,Computer Science,1
2,Mathematics,1
3,Physics,1


In [ ]:
# Execute Aggregate Function query and visualize the result
agg_function_example_2 = """
    SELECT courses.course_name, COUNT (DISTINCT enrollments.student_id )
    AS student_count
    FROM courses
    LEFT JOIN enrollments ON courses.course_id = enrollments.course_id
    GROUP BY courses.course_name
    HAVING student_count > 0;
"""

agg_df_2 = pd.read_sql_query (agg_function_example_2, conn)
agg_df_2

,course_name,student_count
0,Data Structures,1
1,Database Systems,1
2,Linear Algebra,1
3,Quantum Mechanics,1
